<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textaugment
import nltk
nltk.download('all')

     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 42.0 MB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=199eab30c9a5a5c1aad24f004d45cbaec5fd264f635f6b47815c48ef217bfc0b
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping t

True

In [2]:
import pandas as pd
import numpy as np
from textaugment import Wordnet, EDA, word2vec, Translate
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
import re
import matplotlib.pyplot as plt
import seaborn as sns 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv', 'r') as f:
    df = pd.read_csv(f)

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [5]:
df.sub_event_type.value_counts()

Peaceful protest                       329399
Armed clash                            239828
Attack                                 132202
Shelling/artillery/missile attack       64743
Mob violence                            42079
Violent demonstration                   41003
Air/drone strike                        34020
Protest with intervention               31784
Remote explosive/landmine/IED           30620
Looting/property destruction            24315
Abduction/forced disappearance          14624
Arrests                                  8668
Change to group/activity                 7925
Disrupted weapons use                    7499
Government regains territory             6148
Other                                    5353
Excessive force against protesters       3098
Grenade                                  2948
Non-state actor overtakes territory      2670
Sexual violence                          2196
Agreement                                1460
Non-violent transfer of territory 

In [6]:
df.drop_duplicates(['lemmatized']).sub_event_type.value_counts()

Peaceful protest                       305383
Armed clash                             96231
Attack                                  65363
Shelling/artillery/missile attack       33279
Violent demonstration                   24574
Mob violence                            21098
Air/drone strike                        18620
Remote explosive/landmine/IED           16110
Protest with intervention               15003
Looting/property destruction            12036
Abduction/forced disappearance           7214
Change to group/activity                 7098
Arrests                                  4238
Disrupted weapons use                    3866
Other                                    3698
Government regains territory             2424
Grenade                                  1530
Excessive force against protesters       1520
Non-state actor overtakes territory      1198
Sexual violence                          1027
Agreement                                 692
Non-violent transfer of territory 

In [7]:

df.head()

,data_id,event_date,year,event_type,sub_event_type,region,country,location,notes,cleaned,lemmatized,stemmed
0,8378880,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Arnaba,"On 9 July 2021, regime forces shelled Arnaba t...",regime forces shelled Arnaba town in Idleb cou...,regime force shelled arnaba town idleb country...,regim forc shell arnaba town idleb countrysid ...
1,8378881,09-07-2021,2021,Explosions/Remote violence,Shelling/artillery/missile attack,Middle East,Syria,Balyun,"On 9 July 2021, regime forces shelled Balyun t...",regime forces shelled Balyun town and its vici...,regime force shelled balyun town vicinity idle...,regim forc shell balyun town vicin idleb count...
2,8381441,09-07-2021,2021,Protests,Peaceful protest,East Asia,South Korea,Jinju,"On 9 July 2021, local veterans' organizations,...",local veterans organizations including the Kor...,local veteran organization including korean wa...,local veteran organ includ korean war veteran ...
3,8383746,09-07-2021,2021,Riots,Violent demonstration,South Asia,Pakistan,Larkana,"On 9 July 2021, prisoners held a demonstration...",prisoners held a demonstration and set wood an...,prisoner held demonstration set wood clothes f...,prison held demonstr set wood cloth fire larka...
4,8381699,09-07-2021,2021,Violence against civilians,Attack,Southeast Asia,Myanmar,Hpa Yar Gyi,"On 9 July 2021, in Hpa Yar Gyi village (Thongw...",in Hpa Yar Gyi village the village administra...,hpa yar gyi village village administrator shot...,hpa yar gyi villag villag administr shot three...


In [8]:
event_types_dict = {}
for num, name in enumerate(list(df['sub_event_type'].unique())):
    event_types_dict[name] = num
    # print(name, " : ", num)

event_types_dict

event_types_dict_inverse = {k:v for v,k in event_types_dict.items()}
print(event_types_dict)
print( event_types_dict_inverse)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [9]:
def text_to_numerical_category(name):
    num = event_types_dict[name]
    return num

print(text_to_numerical_category('Peaceful protest'))
df['labels'] = df['sub_event_type'].apply(text_to_numerical_category)

1


In [10]:
df.drop(columns=['data_id', 'event_date', 'year','event_type','region','country', 'location',	'notes','cleaned', 'stemmed'	],inplace=True)

In [11]:
from sklearn.utils import shuffle

lemm = []
labels = []
THRESH_1 = 20000
THRESH_2 = 10000

for group in df.sub_event_type.unique():
    if len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_1:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))[:THRESH_1])
        labels += ([group]*THRESH_1)

    elif len(df.groupby('sub_event_type').get_group(group).lemmatized.unique()) > THRESH_2:
        lemm += (list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique())))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized.unique()))))
    else:
        lemm += list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))
        labels += ([group]*len(list(shuffle(df.groupby('sub_event_type').get_group(group).lemmatized))))


In [12]:
len(lemm)

246320

In [13]:
len(labels)

246320

In [14]:
balanced_df = pd.DataFrame({'Text' : lemm, 'Labels': labels})

In [15]:
balanced_df.head(n=10)

,Text,Labels
0,december reported taliban militant planting mi...,Shelling/artillery/missile attack
1,december osce smm observed total burst fire in...,Shelling/artillery/missile attack
2,regime force shelled kadoura idleb using artil...,Shelling/artillery/missile attack
3,april regime force shelled town ankawi neither...,Shelling/artillery/missile attack
4,prohadi islahaffiliated police military force ...,Shelling/artillery/missile attack
5,july antihouthi national resistance force repo...,Shelling/artillery/missile attack
6,rocket fired palestinian militant gaza hit cit...,Shelling/artillery/missile attack
7,january antihouthi force reportedly targeted h...,Shelling/artillery/missile attack
8,march prohouthi force reportedly fired zilzal ...,Shelling/artillery/missile attack
9,regime ground force fired rocket shell town ba...,Shelling/artillery/missile attack


In [16]:
event_types_dict_new = {}
for num, name in enumerate(list(balanced_df.Labels.unique())):
    event_types_dict_new[name] = num
    # print(name, " : ", num)

event_types_dict_new

event_types_dict_inverse_new = {k:v for v,k in event_types_dict_new.items()}
print(event_types_dict_new)
print( event_types_dict_inverse_new)

{'Shelling/artillery/missile attack': 0, 'Peaceful protest': 1, 'Violent demonstration': 2, 'Attack': 3, 'Air/drone strike': 4, 'Armed clash': 5, 'Mob violence': 6, 'Protest with intervention': 7, 'Excessive force against protesters': 8, 'Looting/property destruction': 9, 'Other': 10, 'Remote explosive/landmine/IED': 11, 'Government regains territory': 12, 'Abduction/forced disappearance': 13, 'Change to group/activity': 14, 'Non-state actor overtakes territory': 15, 'Arrests': 16, 'Sexual violence': 17, 'Disrupted weapons use': 18, 'Non-violent transfer of territory': 19, 'Grenade': 20, 'Headquarters or base established': 21, 'Suicide bomb': 22, 'Agreement': 23, 'Chemical weapon': 24}
{0: 'Shelling/artillery/missile attack', 1: 'Peaceful protest', 2: 'Violent demonstration', 3: 'Attack', 4: 'Air/drone strike', 5: 'Armed clash', 6: 'Mob violence', 7: 'Protest with intervention', 8: 'Excessive force against protesters', 9: 'Looting/property destruction', 10: 'Other', 11: 'Remote explosi

In [17]:
balanced_df['NumLabel'] = balanced_df.Labels.apply(text_to_numerical_category)

In [18]:
ed=balanced_df.head(n=25000)


In [19]:
ed.iloc(1)[2][0]

0

In [20]:
sentences = []
for i in range(0,len(ed)):
  sentences.append(ed.iloc(1)[0][i])

len(sentences)

25000

In [21]:
nums = []
for i in range(0,len(ed)):
  nums.append(ed.iloc(1)[2][i])
len(nums)

25000

In [22]:

set(stopwords.words('english')) 

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [31]:
def word_extraction(sentence):    
  ignore = ['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves']    
  words = re.sub("[^\w]", " ",  sentence).split()    
  cleaned_text = [w.lower() 
  for w in words if w not in ignore]    
  return cleaned_text

In [24]:
# def tokenize(sentences):    
#   words = [] 
#   print(len(sentences))   
#   for sentence in sentences:   
#     #print((sentence))     
#     w = word_extraction(sentence)        
#     words.extend(w)            
#     words = sorted(list(set(words))) 

#   return words

In [25]:
# words = tokenize(sentences)

25000


In [26]:
# len(words)

20142

In [ ]:
#import numpy
# new_balanced_df = []
# for sentence in sentences:        
#   words_temp = word_extraction(sentence)        
#   bag_vector = numpy.zeros(len(words))        
#   for w in words_temp:            
#     for i,word in enumerate(words):                
#       if word == w:                     
#         bag_vector[i] += 1                            
#   new_balanced_df.append(numpy.array(bag_vector))

In [28]:
max_classes = len(balanced_df.Labels.unique())

**Model training**

Split into x test and y **test** **bold text**

In [33]:
max_classes = len(balanced_df.Labels.unique())

In [34]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
result = shuffle(balanced_df)

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(balanced_df.Text, balanced_df.NumLabel, test_size = 0.1, random_state = 42, stratify =balanced_df.NumLabel)


In [36]:
len(X_test)

24632

In [37]:
new_corpus = []
for i in range(0,len(X_train)):
  new_corpus.append(X_train.iloc[i])
len(new_corpus)


221688

In [ ]:
new_corpus2 = []
for i in range(0,len(X_test)):
  new_corpus2.append(X_test.iloc[i])
len(new_corpus2)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)

In [41]:

X = cv.fit_transform(new_corpus).toarray()
X_train = X

In [42]:
var = cv.fit_transform(new_corpus2).toarray()
X_test = var

In [43]:
len(X_train[0])

3000

In [45]:
X_train2 = X_train
Y_train2 = Y_train

In [ ]:
# X_train2 = X_train[0:100000]
# Y_train2 = Y_train[0:100000]

In [46]:
X_test2 = X_test
Y_test2 = Y_test

In [ ]:
# X_test2 = X_test[0:10000]
# Y_test2 = Y_test[0:10000]

In [47]:
len(X_train2[0])

3000

**MODEL TRAINING**

Navie Bayes 

In [48]:
# train a Gaussian Naive Bayes classifier on the training set
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, Y_train)

GaussianNB()

In [50]:
Y_pred = gnb.predict(X_test)
from sklearn import metrics
print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(Y_test, Y_pred))


ACCURACY OF THE MODEL:  0.09199415394608638


In [51]:
print(Y_pred)
print(Y_test)

[ 3  9  8 ... 10  2  2]
20079      1
182905    11
173883    10
58362      2
144977     7
          ..
128840     6
190414    12
197173    13
194072    12
93131      4
Name: NumLabel, Length: 24632, dtype: int64


In [52]:
from sklearn.metrics import classification_report

print(classification_report(y_true= Y_test, y_pred= Y_pred))

              precision    recall  f1-score   support

           0       0.42      0.00      0.00      2000
           1       0.19      0.05      0.08      2000
           2       0.11      0.25      0.15      2000
           3       0.11      0.17      0.13      2000
           4       0.04      0.00      0.00      1863
           5       0.16      0.15      0.16      2000
           6       0.10      0.08      0.09      2000
           7       0.09      0.17      0.12      1501
           8       0.00      0.01      0.00       310
           9       0.07      0.13      0.09      1204
          10       0.02      0.15      0.04       535
          11       0.28      0.08      0.12      1611
          12       0.00      0.00      0.00       615
          13       0.36      0.07      0.11      1462
          14       0.03      0.04      0.03       792
          15       0.06      0.00      0.01       267
          16       0.08      0.08      0.08       867
          17       0.08    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K FOLD 

In [53]:
from sklearn import metrics, cross
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
clf, X_train, Y_train, cv=5, scoring='f1_macro')

ImportError: ignored

In [ ]:
#results = model_selection.cross_val_score(clf, cv=kfold)

PRECISION SCORE

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=42)
model=RandomForestClassifier(n_estimators=50) 

results = model_selection.cross_val_score(estimator=model,
                                          X=X_test,
                                          y=Y_test,
                                          cv=kfold,
                                          scoring=scoring)